'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3
SQLALCHEMY VERSION: 1.1.13

'''

In [1]:
import winsound

'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    TableName:Lambda Function
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function can take on any form but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
               CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['NEL_Column'],
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}


import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
#will insert fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)


#for now, since we're developing, delete out all except 1st 2 facilities.
g = session.query(Facility_Chars).filter(Facility_Chars.id >2).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
session.commit #we chose not to synch session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-5
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-6
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-7
Adding to variable dictionary: Count_CB

Reading csv record: Feas-8
Adding to variable dictionary: Runoff_Type

Reading csv record: Feas-9
Adding to variable dictionary: TFMR_Exist

Reading csv record: Feas-10
Adding to variable dictionary: DS_SS_Exist

Reading csv record: Feas-11
Adding to variable dictionary: Fac_Slope

Reading csv record: Feas-12
Adding to variable dictionary: Can_Add_SD

Reading csv record: Feas-13
Adding to variable dictionary: Pave_Area
Adding to variable dictionary: BMP_Size(bas

imported records in  238  rows
associating records...


In [4]:
# EVALUATE BASE BMP FEASIBILITY at each facility
# Write results to the base_bmp_feasibility_test_results table.

print('\n******Evaluating Base BMP feasibility at facilities.******')
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT
for aFac in session.query(Facility_Chars):
    print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        print ('\nEvaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id)
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')


******Evaluating Base BMP feasibility at facilities.******

***Evaluating base bmp feasibiilty tests for facility:  Kalihi-Palama Bus & Paratransit Facility

Evaluating feasibility of base_bmp:  Hydrodynamic Separation  ID:  1

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1

  Attempting eval of feasibility_test ID:  3
proccessing expression: Feas-3=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibi

  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 16

Evaluating feasibility of base_bmp:  Bioinfiltration / Bioretention  ID:  5

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 17

  Attempting eval of feasibility_test ID:  2
proccessing expression: Feas-2=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 18

  Att

proccessing expression: Feas-16=EM_Area>0
    attempting to retrieve value for:  ('EM_Area', ['EM_Area', 'val', 'facility_chars', 'EM_Area', 'id', 'FLOAT'])
       QUERY RESULT: EM_Area=200.0
  eval(200.0>0)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 34

Evaluating feasibility of base_bmp:  Paving and Curbing  ID:  10

  Attempting eval of feasibility_test ID:  17
proccessing expression: Feas-17=Dmg_Pave>0
    attempting to retrieve value for:  ('Dmg_Pave', ['Dmg_Pave', 'val', 'facility_chars', 'Dmg_Pave', 'id', 'FLOAT'])
       QUERY RESULT: Dmg_Pave=0
  eval(0>0)=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 35

Evaluating feasibility of base_bmp:  Oil and Water Separators  ID:  11

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE

    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Silty Clay Loam'
  eval('Silty Clay Loam'!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 52

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=854990.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expressi

       QUERY RESULT: OFFSITE_SD_Exist='No'
  eval('No'=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 66

  Attempting eval of feasibility_test ID:  2
proccessing expression: Feas-2=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 67

  Attempting eval of feasibility_test ID:  5
proccessing expression: Feas-5=Soil_Type!='Rock'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Silty Clay Loam'
  eval('Silty Clay Loam'!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 68

  Attempting eval of 

  Wrote to base_bmp_feasibility_test_results as recordID: 82

Evaluating feasibility of base_bmp:  Roofing  ID:  9

  Attempting eval of feasibility_test ID:  16
proccessing expression: Feas-16=EM_Area>0
    attempting to retrieve value for:  ('EM_Area', ['EM_Area', 'val', 'facility_chars', 'EM_Area', 'id', 'FLOAT'])
       QUERY RESULT: EM_Area=1000.0
  eval(1000.0>0)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 83

Evaluating feasibility of base_bmp:  Paving and Curbing  ID:  10

  Attempting eval of feasibility_test ID:  17
proccessing expression: Feas-17=Dmg_Pave>0
    attempting to retrieve value for:  ('Dmg_Pave', ['Dmg_Pave', 'val', 'facility_chars', 'Dmg_Pave', 'id', 'FLOAT'])
       QUERY RESULT: Dmg_Pave=1
  eval(1>0)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 84

Evaluating feasibility of base_bmp:  Oil and Water Separators  ID:  11

  Attem

*****************************************************************
* Completed evaluating Base BMP feasibility                     *
No errors detected.
*****************************************************************


In [5]:
#CREATE COMBO BMPS USING BASE BMPS
#ALL POSSIBLE COMBOS WILL BE CREATED AND ADDED TO THE COMBO_BMPS TABLE
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT

import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [23]:
'''
Identify the feasible bmp combinations for each facility
Use base bmp feasibility results for each facility.
Put results into the combo_bmp_feasibility_test_results table
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #calculate wrs reduction, insert/update database

    #return as pandas    
    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind) 

    #use information in pd_rr to get CBFTR_record - make new record if necessary
    myCBFTR = Base.metadata.tables['combo_bmp_feasibility_test_results']
    myCBFTR_id = SQLA_main.insertupdateRec(myCBFTR,{'facility_id':myFacility.id, 'combo_bmps_id':pd_rr['combos_bmp_id'][0]},
                and_(
        myCBFTR.c['facility_id'] == myFacility.id,
        myCBFTR.c['combo_bmps_id'] == pd_rr['combos_bmp_id'][0]
                    ))
    session.flush()
    
    print (myCBFTR_id)
    
    #calculate WRS reduction
    myFac_exWRSData = session.query(
    
    

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
#     sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': 'calc_om_cost'}])

    #merge combo bmp's removal rates and costs into 1 dataframe
    return pd.concat([pd_rr, pd_sums], axis = 1)
    
    

def Eval_FacBMPOptions(myFacility):
    #a wrapper around Eval_FacBMPCombo
    #this is used for testing purposes only.

    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    display (df)   
    print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
    df = df.loc[df['is_feasible'] == 1]
    display (df)
    feas_ls = df.index #send feasible base bmp ids to list
#     print (feas_ls)
# from SQLA_DB_combo_bmps import Combo_BMPs
# from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            print ('Here is a summary of the combo: ', list(combo))
            display(Eval_FacBMPCombo(df,myFacility, list(combo)))

            
def Eval_All_FacBMPOptions():
    print ('Evaluating feasibile BMP Options for each facility:')
    for aFac in session.query(Facility_Chars):
        Eval_FacBMPOptions(aFac)
    
Eval_All_FacBMPOptions()
session.commit()

Evaluating feasibile BMP Options for each facility:

***Evaluating feasible bmp combos for facility:  Pearl City Bus Facility ***
****Evaluating feasibile base bmps****


base_bmp_name  calc_cip_cost  \
base_bmp_id                                                                     
1                                      Hydrodynamic Separation            NaN   
2            Enhanced Media Filtration (Replaceable Cartridge)            NaN   
3                                        Biofiltration (Vault)            NaN   
4                                  Media Filtration (Pressure)            NaN   
5                               Bioinfiltration / Bioretention            NaN   
6                                            Inlet Insert Unit         5000.0   
7                                              Sand Filtration            NaN   
8                               Coagulation Enhanced Treatment            NaN   
9                                                      Roofing       220000.0   
10                                          Paving and Curbing            3.0   
11                                    Oil and Water Separators            NaN   
12                                           Vegetative Swales            NaN   
13                                                   Detention            NaN   

             calc_om_cost  is_feasible  
base_bmp_id                             
1                     NaN            0  
2                     NaN            0  
3                     NaN            0  
4                     NaN            0  
5                     NaN            0  
6                  5400.0            1  
7                     NaN            0  
8                     NaN            0  
9                     0.0            1  
10                    0.0            1  
11                    NaN            0  
12                    NaN            0  
13                    NaN            0

****These are the feasible base bmps. I'll use them to make combos:****


base_bmp_name  calc_cip_cost  calc_om_cost  is_feasible
base_bmp_id                                                              
6             Inlet Insert Unit         5000.0        5400.0            1
9                       Roofing       220000.0           0.0            1
10           Paving and Curbing            3.0           0.0            1

Here is a summary of the combo:  [6]
1


...

bmp_fingerprint  combos_bmp_id  PRR_id  r_tss  r_turbidity  r_p  r_n  r_nn  \
0         |6||10|             67      80    0.8          0.0  0.0  0.0   0.0   

   r_an  r_og  r_cu  r_zn  r_fe r_phmin  r_phmax  calc_cip_cost  calc_om_cost  
0   0.0   0.8   0.0   0.0   0.0    None      0.0         5003.0  calc_om_cost

Here is a summary of the combo:  [9, 10]
13


bmp_fingerprint  combos_bmp_id  PRR_id  r_tss  r_turbidity  r_p  r_n  r_nn  \
0         |9||10|             82      95    1.0          1.0  1.0  1.0   1.0   

   r_an  r_og  r_cu  r_zn  r_fe r_phmin  r_phmax  calc_cip_cost  calc_om_cost  
0   1.0   1.0   1.0   1.0   1.0    None      1.0       220003.0  calc_om_cost

Here is a summary of the combo:  [6, 9, 10]
14


bmp_fingerprint  combos_bmp_id  PRR_id  r_tss  r_turbidity  r_p  r_n  r_nn  \
0      |6||9||10|            333     346    1.0          1.0  1.0  1.0   1.0   

   r_an  r_og  r_cu  r_zn  r_fe r_phmin  r_phmax  calc_cip_cost  calc_om_cost  
0   1.0   1.0   1.0   1.0   1.0    None      1.0       225003.0  calc_om_cost

In [ ]:
# session.close()
# engine.dispose()

In [ ]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))
